# Table 2 
Internal Training/Optimal Model Selection - Performance summary of Decision Trees Classifier on F1||F5 and F5/F1 Delta Radiomics Models with and without Auxiliary Data.

# Table 3
External Testing -  Performance Summary of Decision Trees Classifier on the External Adrenal data.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,ParameterGrid
from sklearn.linear_model import Lasso, Ridge, ElasticNet
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,classification_report
from sklearn.model_selection import KFold,cross_val_score,cross_validate, cross_val_predict,RepeatedKFold, StratifiedKFold,RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline
from random import randrange, uniform
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE,SVMSMOTE,SMOTEN
from sklearn.neighbors import NearestNeighbors
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Load Dataset

AdrenalData = pd.read_excel("Path to Internal Adrenal Dataset Delta features")
LungData = pd.read_excel("Path to Internal Lung Dataset Delta features")
PancreasData = pd.read_excel("Path to Internal Pancreas Dataset Delta features")

# Define the target and feature vectors in the dataset

Adrenal_Ytrain = AdrenalData.iloc[:,0] # use the slicing value that applies to your data
Adrenal_Xtrain = AdrenalData.iloc[:,1:]

Lung_Ytrain = LungData.iloc[:,1]
Lung_Xtrain = LungData[:,2:]


pancreas_Ytrain = PancreasData.iloc[:,1]
pancreas_Xtrain = PancreasData.iloc[:,2:]


In [ ]:
# functions

# Bootstrap Function
def Bootstrap(x, y, n_split, random_seed=42):
    import numpy as np
    np.random.seed(42)
    rng = np.random.RandomState(random_seed)
    sample_idx = np.arange(x.shape[0])
    set_idx = set(sample_idx)
    Train_id={}
    Test_id={}
    for i in range(n_split):
        while True:
            train_idx= rng.choice(sample_idx[y==1],
                                   size=np.sum(y),
                                   replace=True)
            train_idx= np.append(train_idx, rng.choice(sample_idx[y==0],
                                   size=np.sum(1-y),
                                   replace=True))
            
            if len(set(sample_idx[y==1]).difference(set(train_idx))) and len(set(sample_idx[y==0]).difference(set(train_idx))):
                break
        test_idx = np.array(list(set_idx - set(train_idx)))
        Train_id[i]=train_idx
        Test_id[i]=test_idx
            
    return Train_id, Test_id  
     

In [ ]:
# Parameter Grid for Feature Selection and Hyperparameter Tuning

# Feel free to modify

# ElasticNet Feature Selection Parameter Grid and Pipleline
param_grid_EN = {
    'model__alpha': [0.1],
    'model__l1_ratio':[0.4],
}

pipeline_EN = Pipeline([
            ('scaler', StandardScaler()),
            ('model', ElasticNet())
        ])


# Machine Learning Classifiers

classifiers = {
    "DT": DecisionTreeClassifier()     
    }

# Classifier Pipelines

pipelines = {}
for name, classifier in classifiers.items():
    pipeline = Pipeline([
        ("classifier", classifier)
    ])
    
    pipelines[name] = pipeline

# Parameter grids for classifiers
params = {
    "DT": {
        'classifier__max_depth': np.arange(2, 10, 1),
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__min_samples_split': np.arange(2, 5, 1),
        'classifier__min_samples_leaf': np.arange(2, 5, 1),
        'classifier__random_state': [0,1,10,42],
        
    },
}

# Model Development 





In [ ]:
# Divide the Adrenal Data into 42 Bootstrapped Train and Test folds

Train_id, Test_id = Bootstrap(Adrenal_Xtrain, Adrenal_Ytrain, n_split=42, random_seed=42)

# Define inner cross-validation fold

inner_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Feature selection, Hyperparameter tuning and best estimator selection.

best_estimators = []  # List of the best estimators after hyperparameter tuning


for i in range(len(Train_id)):
    X_train, y_train = Adrenal_Xtrain.iloc[Train_id[i]], Adrenal_Ytrain.iloc[Train_id[i]]
    X_test, y_test = Adrenal_Xtrain.iloc[Test_id[i]], Adrenal_Ytrain.iloc[Test_id[i]]
    
    # Feature Selection
    # Note that feature selection is done only on the Adrenal Data and not on auxiliary data.
    search = GridSearchCV(pipeline_EN,
                      param_grid_EN,
                      cv=inner_kf,
                      scoring="neg_root_mean_squared_error")

    search.fit(X_train, y_train)

    # Get the best parameters
    best_parameter = search.best_params_
    coefficients = search.best_estimator_.named_steps['model'].coef_
    selected_features = np.array(features)[coefficients != 0]
    print(selected_features)
    print("####################################################################################")


    
    # Concatenate the Auxiliary Data after feature selection
    
    # Pancreas
    XX_train = pd.concat([X_train, pancreas_Xtrain])
    yy_train = pd.concat([y_train, pancreas_Ytrain])
    
    # OR
    
    # Lung
    XX_train = pd.concat([X_train, lung_Xtrain])
    yy_train = pd.concat([y_train, lung_Ytrain])
    
    
    
    # Use the selected features for further processing
    Xtrain_hyper = XX_train[selected_features]
    
    # Xvalid is the Adrenal Data that is reserved for testing the best estimator from the hyperparameter tuning.
    Xvalid = X_test[selected_features] 
    
    # Data Augmentation - BorderlineSMOTE-1 
    bsmote = BorderlineSMOTE(random_state=10, kind='borderline-1',k_neighbors=3, sampling_strategy='minority')
    X_resampled, y_resampled = bsmote.fit_resample(Xtrain_hyper, y_train)
    
    
    # Hyperparameter Tuning
    grid_kf = {}
    for name, pipeline in pipelines.items():
        grid_search_inner_kf = GridSearchCV(estimator=pipeline,
                                            param_grid=params[name],
                                            scoring="roc_auc",
                                            n_jobs=-1,
                                            cv=inner_kf,
                                            verbose=0,
                                            refit=True)
        grid_kf[name] = grid_search_inner_kf

    for name, grid_search in grid_kf.items():
        grid_kf[name].fit(X_resampled, y_resampled)
        best_estimator = grid_kf[name].best_estimator_
        ypred = grid_kf[name].best_estimator_.predict(Xvalid_hyper)
        
        # Check for empty and unique predictions
        if ypred is not None:  
            unique_classes = np.unique(ypred)
            if len(unique_classes) == 1:
                print(f"Skip: Estimator from '{name}' predicts only one class.")
                continue  # Skip to the next iteration

            score = roc_auc_score(ypred, y_test)
            print("estimator:", best_estimator)
            print('estimator_score: ', score)
            print("-" * 100)
            # Save the best estimator, its score and the selected features in the outer list- best estimator
        best_estimators.append({'Algorithm' : name, 'selected_features': selected_features, 'estimator': best_estimator, 'score': score})


# Model Evaluation

In [ ]:
# Filter best estimators of each classifier's results
classifier_results = [estimator for estimator in best_estimators if estimator['Algorithm'] == 'DT']

# best estimator is the estimator with the highest AUC on the Adrenal validation fold
best_estimator = max(classifier_results, key=lambda x: x['score'])
best_features = best_estimator_score['selected_features']
clf = best_estimator['estimator']

#Update Data

Adrenal_Xtrain = Adrenal_Xtrain[best_features]
lung_Xtrain = lung_Xtrain[best_features]
pancreas_Xtrain = pancreas_Xtrain[best_features]

# Correlation Matrix
corr_matrix = Adrenal_Xtrain.corr(method='pearson')
sns.set(style="white")
plt.figure(figsize=(20,20)) 
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')

# Model Evaluation 

model_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

outer_scores_auc = []
outer_scores_acc = []
outer_scores_f1 = []
outer_scores_ppv = []
outer_scores_npv = []

for train_idx, valid_idx in model_kf.split(Adrenal_Xtrain, Adrenal_Ytrain):
    X_train, X_valid = Adrenal_Xtrain.iloc[train_idx], Adrenal_Xtrain.iloc[valid_idx]
    y_train, y_valid = Adrenal_Ytrain.iloc[train_idx], Adrenal_Ytrain.iloc[valid_idx]
    
    # Concatenate Auxiliary Data
    X_train = pd.concat([X_train, lung_Xtrain])
    y_train = pd.concat([y_train, lung_Ytrain])
    
    X_train = pd.concat([X_train, pancreas_Xtrain])
    y_train = pd.concat([y_train, pancreas_Ytrain])
    

    bsmote = BorderlineSMOTE(random_state=10, kind='borderline-1',k_neighbors=3, sampling_strategy='minority')
    X_resampled, y_resampled = bsmote.fit_resample(X_train, y_train)

    clf.fit(X_resampled, y_resampled)
    y_pred = clf.predict(X_valid)
    
    
     # metrics
    # ROC_AUC
    AUC_X = roc_auc_score(y_valid,y_pred)
    outer_scores_auc.append(AUC_X)
    
    #Accuracy
    ACC_X = accuracy_score(y_valid,y_pred)
    outer_scores_acc.append(ACC_X)
    
    #f1-score
    f1_X = f1_score(y_valid,y_pred)
    outer_scores_f1.append(f1_X)
    
    #Confusion Matrix, PPV, NPV
    cm = confusion_matrix(y_valid,y_pred)
    TN,FP,FN,TP = cm.ravel()
    #PPV
    PPV_X = TP / (TP + FP)
    outer_scores_ppv.append(PPV_X)
    #NPV
    NPV_X = TN / (TN + FN)
    outer_scores_npv.append(NPV_X)
    
    
print("AUC")   
# AUC   
print(np.mean(outer_scores_auc))
SD_auc = np.std(outer_scores_auc)
print(outer_scores_auc)
print(SD_auc)

print("*"*100)

print("ACC")

# ACC   
print(np.mean(outer_scores_acc))
SD_acc = np.std(outer_scores_acc)
print(outer_scores_acc)
print(SD_acc)

print("*"*100)

print("f1-score")

#f1-score
# AUC   
print(np.mean(outer_scores_f1))
SD_f1 = np.std(outer_scores_f1)
print(outer_scores_f1)
print(SD_f1)


print("*"*100)

print("PPV")

# PPV 
print(np.mean(outer_scores_ppv))
Sd_ppv = np.std(outer_scores_ppv)
print(outer_scores_ppv)
print(SD_ppv)


print("*"*100)

print("NPV")  
print(np.mean(outer_scores_npv))
SD_npv = np.std(outer_scores_npv)
print(outer_scores_npv)
print(SD_npv)

# External Validation

In [ ]:
# Load The data

ExternalData = pd.read_excel("Path to the Delta features of the external dataset")

External_Xtest = ExternalData[best_features]
External_Ytest = ExternalData.iloc[:,1]



AdLung_Xtrain = pd.concat([Adrenal_Xtrain, lung_Xtrain])
AdLung_Ytrain = pd.concat([Adrenal_Ytrain, lung_Ytrain])

bsmote = BorderlineSMOTE(random_state=10, kind='borderline-1', k_neighbors=3, sampling_strategy='minority')
X_resampled, y_resampled = bsmote.fit_resample(AdLung_Xtrain,AdLung_Ytrain)

clf_DT.fit(X_resampled, y_resampled)

y_pred = clf_DT.predict(mci_Xtest)


print("*"*100)
# ROC_AUC
auc_score = roc_auc_score(External_Ytest, y_pred)
print("External AUC: ", auc_score)

print("*"*100)

#Accuracy
ACC = accuracy_score(y_pred,External_Ytest)
print("External ACC: ", ACC)

print("*"*100)

#f1-score
f1 = f1_score(y_pred,External_Ytest)
print("External f1-score : ", f1)

print("*"*100)

#Confusion Matrix, PPV, NPV
cm = confusion_matrix(y_pred,External_Ytest)
TN,FP,FN,TP = cm.ravel()
#PPV
PPV = TP / (TP + FP)
print("External PPV: ", PPV)

print("*"*100)

#NPV
NPV = TN / (TN + FN)
print("External NPV: ", NPV)

print("*"*100)